In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Heuristic feature model

We test a model which predicts participants responses based on the feature of the task, rather than on a ToM model. The features where:
- Agent certainty (as color so as a factor): purple, yellow, green
- Agent face: sad - happy

We use the training data to estimate the probability of each response given the features. We then use these probabilities to predict the responses on the test data.

We assume that people are trying to jointly optimize boss response and the face of the other, i.e. take the path which minimizes the length but also keeps the other happy

The model chooses path $P$ with a probability:
$$
    p(P|c, f, b) = \exp( a_0 c + a_1 f + a_2 b)
$$
Then we can try more complex versions of the model with interactions.
As $c$ is a factor, we need three parameters.

In [4]:

df_raw = pd.read_csv('data/pilot/raw/20_01_2025/tom_dictator_v2_2025-01-20.csv')
                     
# Replace full stop with underscore in df_raw column names
df_raw.columns = df_raw.columns.str.replace('.', '_')

sessions = [
    'tgvqauld',
    'ta176utg'
]

# Only keep rows where session_id is in sessions and where participant_label is not NaN
df_full = df_raw[df_raw['session_code'].isin(sessions) & df_raw['participant_label'].notna() & (df_raw['participant__current_page_name'] == 'End')]
# Insert 
columns = ['participant_code', 'participant_label', 'session_code',
       
       'player_block_type', 'player_block_idx', 'player_idx_in_block', 'player_trust_cond', 'player_trust_label',
       'player_intention', 'player_interest', 
       'player_certainty', 'player_certainty_val', 'player_rigidity', 'player_rigidity_val',
       'player_k_lvl', 'player_trust_predicted', 'player_alpha_prior',
       'player_alpha_prior_entropy', 'player_alpha', 'player_alpha_entropy',
       'player_beta_prior', 'player_beta_prior_entropy', 'player_beta',
       'player_beta_entropy', 'player_beta_social_prior',
       'player_beta_social_prior_entropy', 'player_beta_social',
       'player_beta_social_entropy', 'player_path_taken',
       'player_perceived_certainty_certain',
       'player_perceived_certainty_uncertain',
       'player_perceived_certainty_immutable', 'player_attention',
       'player_attention_passed', 'player_attention_correct',

       'player_focus_work_high', 'player_focus_work_medium', 'player_focus_work_low',
       'player_strategy_high', 'player_strategy_low', 'player_strategy_medium',

       'subsession_round_number',
       'participant__current_app_name',
       'participant__current_page_name', 
       'player_payoff',
]


df = df_full[columns]

# If player is in column name, remove player_ from column name
df.columns = df.columns.str.replace('player_', '')


df_long = df[(df.block_type != 'test')][['participant_code', 'intention', 'certainty', 'rigidity', 'intention', 'trust_cond', 'path_taken']]

df_long['absent'] = df_long.trust_cond.map(lambda x: x == 'none')

# Get dummies for path taken and concat with df_long
path_taken_dummies = pd.get_dummies(df_long['path_taken'], prefix='', prefix_sep='')
df_long = pd.concat([df_long, path_taken_dummies], axis=1)

df_long['delivered'] = df_long['A'] + df_long['B'] > 0
df_long['path_hidden'] = df_long['B'] + df_long['C'] > 0

# Create path_idx column
path_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
}
df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
df_long['path_length'] = df_long['path_taken'].map(lambda x: path_dict[x])

# Create certainty_lvl column
certainties_dict = {
    'uncertain': 1,
    'certain': 2,
    'immutable': 3,
}
df_long['certainty_lvl'] = df_long['certainty'].map(lambda x: certainties_dict[x])
df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])

# Create trust_lvl column
trust_dict = {
    'none': 4,
    'low': 1,
    'medium': 2,
    'high': 3,
}
df_long['trust_lvl'] = df_long['trust_cond'].map(lambda x: trust_dict[x])
df['trust_lvl'] = df['trust_cond'].map(lambda x: trust_dict[x])

# Assign unique integer to each participant and make sure it is the same for both df and df_long and recoverable
df_long['part_idx'] = pd.Categorical(df_long['participant_code']).codes
df['part_idx'] = pd.Categorical(df['participant_code']).codes



paths_colors = {
    'A': np.array([238, 154, 86])/255,
    'B': np.array([183, 117, 63])/255,
    'C': np.array([50, 114, 169])/255,
    'D': np.array([88, 182, 225])/255,
}

paths_colors_next = {
    'A': np.array([183, 117, 63])/255,
    'B': np.array([50, 114, 169])/255,
    'C': np.array([88, 182, 225])/255,
}

/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_1599/955521442.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['path_idx'] = df['path_taken'].map(lambda x: path_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_1599/955521442.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['certainty_lvl'] = df['certainty'].map(lambda x: certainties_dict[x])
/var/folders/hh/c12d0kb97ln3mqms4k45kqwm0000gn/T/ipykernel_1599/955521442.py:84: SettingWithCopyWarning: 
A value is trying

In [5]:
df_long

,participant_code,intention,certainty,rigidity,intention,trust_cond,path_taken,absent,A,B,C,D,delivered,path_hidden,path_length,certainty_lvl,trust_lvl,part_idx
66,ivf2doqd,none,uncertain,bayesian,none,medium,C,False,False,False,True,False,False,True,3,1,2,54
68,hh2ezh9j,none,uncertain,bayesian,none,medium,B,False,False,True,False,False,True,True,2,1,2,48
69,g3l64f2k,none,uncertain,bayesian,none,medium,B,False,False,True,False,False,True,True,2,1,2,42
70,qwthoytg,none,uncertain,bayesian,none,medium,A,False,True,False,False,False,True,False,1,1,2,74
72,81qc8udv,none,uncertain,bayesian,none,medium,B,False,False,True,False,False,True,True,2,1,2,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10394,fc2ylfo4,none,immutable,bayesian,none,high,D,False,False,False,False,True,False,False,4,3,3,41
10395,96sj342h,none,certain,bayesian,none,medium,A,False,True,False,False,False,True,False,1,2,2,24
10396,n626q41f,none,certain,bayesian,none,medium,A,False,True,False,False,False,True,False,1,2,2,60
10397,cppwbnz4,none,certain,bayesian,none,low,B,False,False,True,False,False,True,True,2,2,1,34
